In [1]:
import pandas as pd

In [2]:
from pulp import *

In [3]:
data = pd.read_excel("diet.xls", header = 0)

In [4]:
#eliminating rows with no food data
data = data[:-3]

In [5]:
data.head()

,Foods,Price/ Serving,Serving Size,Calories,Cholesterol mg,Total_Fat g,Sodium mg,Carbohydrates g,Dietary_Fiber g,Protein g,Vit_A IU,Vit_C IU,Calcium mg,Iron mg
0,Frozen Broccoli,0.16,10 Oz Pkg,73.8,0.0,0.8,68.2,13.6,8.5,8.0,5867.4,160.2,159.0,2.3
1,"Carrots,Raw",0.07,1/2 Cup Shredded,23.7,0.0,0.1,19.2,5.6,1.6,0.6,15471.0,5.1,14.9,0.3
2,"Celery, Raw",0.04,1 Stalk,6.4,0.0,0.1,34.8,1.5,0.7,0.3,53.6,2.8,16.0,0.2
3,Frozen Corn,0.18,1/2 Cup,72.2,0.0,0.6,2.5,17.1,2.0,2.5,106.6,5.2,3.3,0.3
4,"Lettuce,Iceberg,Raw",0.02,1 Leaf,2.6,0.0,0.0,1.8,0.4,0.3,0.2,66.0,0.8,3.8,0.1


In [6]:
#make list
data2 = data.values.tolist()

In [7]:
#make list of foods
foods = [x[0] for x in data2]

In [8]:
#dictionary of for each nutritional value
price = {x[0]:float(x[1]) for x in data2}
calories = {x[0]:float(x[3]) for x in data2}
cholesterol = {x[0]:float(x[4]) for x in data2}
fat = {x[0]:float(x[5]) for x in data2}
sodium = {x[0]:float(x[6]) for x in data2}
carb = {x[0]:float(x[6]) for x in data2}
fiber = {x[0]:float(x[7]) for x in data2}
protein = {x[0]:float(x[8]) for x in data2}
vita = {x[0]:float(x[9]) for x in data2}
vitc = {x[0]:float(x[10]) for x in data2}
calcium = {x[0]:float(x[11]) for x in data2}
iron = {x[0]:float(x[12]) for x in data2}

In [9]:
#set up the optimaization problem
prob = LpProblem('diet', LpMinimize)

#define vars as dicts
lp_vars = LpVariable.dicts("Foods", foods, 0)

#binary for if food is chose or not
chosenVars = LpVariable.dicts("food_chosen",foods,0,1,cat='Binary')

#create objective function
prob += lpSum([price[i] * lp_vars[i] for i in foods]), "Total Cost"


In [10]:
#create constrains
minVal = data[65:66].values.tolist() # minimum nutrient values
maxVal = data[66:67].values.tolist() # maximum nutrient values

for i in range(0,11): # for loop running through each nutrient: 11 times starting with 0
    prob += lpSum([nutrients[i][j] * foodVars[j] for j in foods]) >= minVal[0][i+3], 'min nutrient ' + nutrientNames[i]
    prob += lpSum([nutrients[i][j] * foodVars[j] for j in foods]) <= maxVal[0][i+3], 'max nutrient ' + nutrientNames[i]

for food in foods:
    prob += lp_vars[food] >= 0.1 * foodVars_selected[food]

# If any of a food is eaten, its binary variable must be 1

for food in foods:
    prob += lp

# If a food is eaten, must eat at least 0.1 serving

for food in foods:
    prob += foodVars[food] >= 0.1 * foodVars_selected[food]

# If any of a food is eaten, its binary variable must be 1

for food in foods:
    prob += foodVars_selected[food] >= foodVars[food]*0.0000001

# Include at most 1 of celery and frozen brocolli

prob += foodVars_selected['Frozen Broccoli'] + foodVars_selected['Celery, Raw'] <= 1

# At least 3 kinds of meat/poultry/fish/eggs

prob += foodVars_selected['Roasted Chicken'] + foodVars_selected['Poached Eggs'] \
        + foodVars_selected['Scrambled Eggs'] + foodVars_selected['Bologna,Turkey'] \
        + foodVars_selected['Frankfurter, Beef'] + foodVars_selected['Ham,Sliced,Extralean'] \
        + foodVars_selected['Kielbasa,Prk'] + foodVars_selected['Pizza W/Pepperoni'] \
        + foodVars_selected['Hamburger W/Toppings'] \
        + foodVars_selected['Hotdog, Plain'] + foodVars_selected['Pork'] \
        + foodVars_selected['Sardines in Oil'] + foodVars_selected['White Tuna in Water'] \
        + foodVars_selected['Chicknoodl Soup'] + foodVars_selected['Splt Pea&Hamsoup'] \
        + foodVars_selected['Vegetbeef Soup'] + foodVars_selected['Neweng Clamchwd'] \
        + foodVars_selected['New E Clamchwd,W/Mlk'] + foodVars_selected['Beanbacn Soup,W/Watr'] >= 3

NameError: name 'nutrients' is not defined

In [29]:
# The problem data is written to an .lp file
#prob.writeLP("Diet.lp")

In [30]:
prob.solve()
print("Status:", LpStatus[prob.status])

Status: Infeasible


In [31]:
for i in prob.variables():
    if i.varValue > 0:
        if str(i).find('Chosen'):
            print(str(i.varValue) + " units of " + str(i))
            
# print the costs of the optimal diet
cost = value(prob.objective)
print("Total cost of food = $", round(cost,2))

41472.801 units of Foods_Peppers,_Sweet,_Raw
830.77278 units of Foods_Tofu
364.88161 units of Foods_White_Tuna_in_Water
Total cost of food = $ 20261.33
